<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/10-Optimizacion-Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Escuela de Ciencias Físicas y Matemática</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Optmización de Hiperparámetros
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Andrés Merino &bull; 2025-04
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

A lo largo de este taller, aplicaremos optimización de hiperparámetros en un modelo que elijas.

Los paquetes necesarios son:

In [24]:
# Paquetes necesarios:
import pandas as pd                    # Manejo de datos
import numpy as np                     # Operaciones numéricas
import matplotlib.pyplot as plt        # Visualización

# Preprocesamiento y modelado
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier

import joblib  # Para guardar el modelo

---
## <font color='264CC7'> Clasificación </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero necesitas el conjunto de datos. Los datos a utilizar son los seleccionados en la clase anterior.

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Carga el conjunto de datos y procésalos:
<ul>
  <li>Muestra algunos datos.</li>
  <li>Muestra una descripción de los datos.</li>
  <li>Escala los datos si es necesario.</li>
</ul>
</div>

In [8]:
# Cargar el conjunto de datos desde GitHub
url = "https://raw.githubusercontent.com/Jonathan-Carrillo/Base_Cybersecurity/main/datos_filtrados.csv"
df = pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print("Primeras filas del conjunto de datos:")
display(df.head())

# Mostrar la descripción estadística de las variables numéricas
print("\nDescripción de las variables numéricas:")
display(df.describe())

# Crear variable binaria de clasificación (objetivo)
# Consideraremos 'High_Loss' = 1 si la pérdida financiera está por encima de la mediana
df['High_Loss'] = (df['Financial Loss (in Million $)'] > df['Financial Loss (in Million $)'].median()).astype(int)

# Verificar la distribución de clases
print("\nDistribución de clases (High_Loss):")
display(df['High_Loss'].value_counts())

# Seleccionar las columnas numéricas como variables predictoras y variable objetivo
X = df[['Number of Affected Users', 'Incident Resolution Time (in Hours)', 'Year']]
y = df['High_Loss']

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

Primeras filas del conjunto de datos:


,Number of Affected Users,Incident Resolution Time (in Hours),Country,Year,Financial Loss (in Million $)
0,773169,63,China,2019,80.53
1,295961,71,China,2019,62.19
2,605895,20,India,2017,38.65
3,659320,7,UK,2024,41.44
4,810682,68,Germany,2018,74.41



Descripción de las variables numéricas:


,Number of Affected Users,Incident Resolution Time (in Hours),Year,Financial Loss (in Million $)
count,3000.000000,3000.000000,3000.000000,3000.000000
mean,504684.136333,36.476000,2019.570333,50.492970
std,289944.084972,20.570768,2.857932,28.791415
min,424.000000,1.000000,2015.000000,0.500000
25%,255805.250000,19.000000,2017.000000,25.757500
50%,504513.000000,37.000000,2020.000000,50.795000
75%,758088.500000,55.000000,2022.000000,75.630000
max,999635.000000,72.000000,2024.000000,99.990000



Distribución de clases (High_Loss):


,count
High_Loss,
1,1500
0,1500


**Nota:** Aplicamos la creación de una variable binaria de clasificación (High_Loss), sin escalar los datos (ya que Random Forest no lo requiere)


### <font color='264CC7'> Modelo </font>


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Selecciona el mejor modelo de las clases anteriores.
<ul>
  <li>Muestra los hiperparámetros del modelo.</li>
  <li>Consulta qué significan al menos 4 hiperparámetros.</li>
  <li>Selecciona los hiperparámetros que deseas optimizar, al menos 3.</li>
</ul>
</div>

In [9]:
# Definir modelo base
modelo_base = RandomForestClassifier(random_state=42)

# Mostrar los hiperparámetros disponibles
print("Hiperparámetros del modelo RandomForestClassifier:")
for parametro, valor in modelo_base.get_params().items():
    print(f"{parametro}: {valor}")

Hiperparámetros del modelo RandomForestClassifier:
bootstrap: True
ccp_alpha: 0.0
class_weight: None
criterion: gini
max_depth: None
max_features: sqrt
max_leaf_nodes: None
max_samples: None
min_impurity_decrease: 0.0
min_samples_leaf: 1
min_samples_split: 2
min_weight_fraction_leaf: 0.0
monotonic_cst: None
n_estimators: 100
n_jobs: None
oob_score: False
random_state: 42
verbose: 0
warm_start: False


**Significado de 4 hiperparámetros:**

| Hiperparámetro      | Descripción                                                                                                        |
| ------------------- | ------------------------------------------------------------------------------------------------------------------ |
| `n_estimators`      | Número de árboles en el bosque. Más árboles pueden mejorar la precisión, pero aumentan el tiempo de entrenamiento. |
| `max_depth`         | Profundidad máxima de cada árbol. Limitarla ayuda a prevenir sobreajuste.                                          |
| `min_samples_split` | Número mínimo de muestras necesarias para dividir un nodo. Controla la complejidad del árbol.                      |
| `max_features`      | Número de características a considerar al buscar la mejor división. Reduce la correlación entre árboles.           |

**Hiperparámetros que se optimizaran:**

- n_estimators: número de árboles

- max_depth: profundidad máxima de los árboles

- min_samples_split: mínimo de muestras para dividir un nodo

- max_features: características para mejor división.

### <font color='264CC7'> Optimización por GridSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica GridSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 3 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [31]:
# Definir los valores a probar para cada hiperparámetro
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt', 'log2', None]
}

# Definir validación cruzada de 5 pliegues
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Crear el modelo base
modelo_base = RandomForestClassifier(random_state=42)

# Configurar el GridSearchCV
grid_search = GridSearchCV(
    estimator=modelo_base,
    param_grid=param_grid,
    cv=cv,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Entrenar el modelo
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]},
             scoring='accuracy', verbose=1)

In [32]:
# Mostrar los mejores parámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Mostrar el mejor score promedio durante la validación cruzada
print("\nMejor precisión promedio (cross-validation):")
print(round(grid_search.best_score_, 4))

Mejores hiperparámetros encontrados:
{'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 50}

Mejor precisión promedio (cross-validation):
0.511


- Aunque la precisión no fue alta, el modelo seleccionado es el mejor dentro del espacio de búsqueda definido. Esto indica que posiblemente los datos sean complejos de separar o que se podrían explorar más atributos o técnicas.

### <font color='264CC7'> Optimización por RandomSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica RandomSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 5 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Usa RandomSearchCV con 25 iteraciones.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [29]:
# Definir el modelo base
modelo_base = RandomForestClassifier(random_state=42)

# Hiperparámetros a explorar (mínimo 5 valores por parámetro)
param_distributions = {
    'n_estimators': [50, 100, 150, 200, 250],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10, 15, 20],
    'max_features': ['sqrt', 'log2', None]
}

# Validación cruzada
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=modelo_base,
    param_distributions=param_distributions,
    n_iter=25,
    scoring='accuracy',
    cv=kfold,
    random_state=42,
    verbose=1,
    n_jobs=-1
)

# Entrenar el modelo
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=RandomForestClassifier(random_state=42), n_iter=25,
                   n_jobs=-1,
                   param_distributions={'max_depth': [None, 10, 20, 30, 40],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_split': [2, 5, 10, 15, 20],
                                        'n_estimators': [50, 100, 150, 200,
                                                         250]},
                   random_state=42, scoring='accuracy', verbose=1)

In [30]:
# Mostrar mejores parámetros
print("Mejores hiperparámetros encontrados:")
print(random_search.best_params_)

# Mostrar el mejor score promedio
print("\nMejor precisión promedio (cross-validation):")
print(round(random_search.best_score_, 4))

Mejores hiperparámetros encontrados:
{'n_estimators': 250, 'min_samples_split': 10, 'max_features': 'log2', 'max_depth': None}

Mejor precisión promedio (cross-validation):
0.5076


- A pesar de explorar un amplio rango de combinaciones, la precisión se mantuvo similar a la obtenida con GridSearch, lo cual sugiere que el modelo puede estar limitado por la estructura del conjunto de datos o requiere una transformación adicional de las variables.

### <font color='264CC7'> Guardado de modelo </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Con los parámetros óptimos que mejor resultado dieron, reentrena el modelo, muestra su score y guárdalo.
</div>

In [27]:
# Reentrenar el modelo con los mejores hiperparámetros
modelo_final = RandomForestClassifier(
    n_estimators=250,
    max_depth=None,
    min_samples_split=10,
    max_features='log2',
    random_state=42
)
modelo_final.fit(X_train, y_train)

# Evaluar el modelo con el conjunto de prueba
y_pred = modelo_final.predict(X_test)

# Métricas de evaluación
accuracy = round(accuracy_score(y_test, y_pred), 4)
print(" Precisión del modelo final:", accuracy)
print("\n Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("\n Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

 Precisión del modelo final: 0.4967

 Matriz de Confusión:
[[230 220]
 [233 217]]

 Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.50      0.51      0.50       450
           1       0.50      0.48      0.49       450

    accuracy                           0.50       900
   macro avg       0.50      0.50      0.50       900
weighted avg       0.50      0.50      0.50       900



In [28]:
# Guardar el modelo entrenado
joblib.dump(modelo_final, "modelo_random_forest_optimo.pkl")
print("\n El modelo ha sido guardado como 'modelo_random_forest_optimo.pkl'")


 El modelo ha sido guardado como 'modelo_random_forest_optimo.pkl'


### <font color='264CC7'> Publicación </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Coloca el este cuaderno y el modelo en tu repositorio de GitHub. Agrega una licencia MIT y un README.md donde se explique el contenido del repositorio, los datos utilizados y los resultados obtenidos.
</div>